In [3]:
# ----------------------------
# 1️⃣ Bibliotheken importieren
# ----------------------------
import pandas as pd
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# ----------------------------
# 2️⃣ CSV einlesen
# ----------------------------
# Passe hier den Dateinamen an
CSV_FILE = "20250318_Export_KUBA_EPZ-KernInfO_DE_FR.csv"

df = pd.read_csv(CSV_FILE, sep=";", engine="python")

# ----------------------------
# 3️⃣ Spalten definieren
# ----------------------------
# 0-basierte Indexe
HIER_CODE_COL = 4      # Hierarchiecode (5. Spalte)
TYPE_NAME_COL = 5      # Typ-Name (6. Spalte)
OBJ1_COL = 1           # Objektname 1 (2. Spalte)
OBJ2_COL = 2           # Objektname 2 (3. Spalte)
FILTER_COL = 3         # Attribut für Filterung (7. Spalte), optional

# ----------------------------
# 4️⃣ Hilfsfunktion: Baum aus DataFrame aufbauen
# ----------------------------
def build_tree_from_df(df_filtered):
    tree = defaultdict(list)
    agg_counts = defaultdict(int)
    direct_counts = defaultdict(int)
    code_to_objects = defaultdict(list)
    code_to_name = {}

    for _, row in df_filtered.iterrows():
        code = str(row.iloc[HIER_CODE_COL]).strip()
        name = str(row.iloc[TYPE_NAME_COL]).strip()
        obj = f"{row.iloc[OBJ1_COL]} / {row.iloc[OBJ2_COL]}"

        code_to_name[code] = name
        direct_counts[code] += 1
        code_to_objects[code].append(obj)

        # Hochzählung für übergeordnete Hierarchie
        for i in range(2, len(code)+1):
            parent = code[:i]
            agg_counts[parent] += 1
            if parent != code and code not in tree[parent]:
                tree[parent].append(code)

    return tree, agg_counts, direct_counts, code_to_objects, code_to_name

# ----------------------------
# 5️⃣ Hilfsfunktion: Button für direkte Objekte
# ----------------------------
def direct_objects_button(code, code_to_objects, code_to_name):
    objs = code_to_objects.get(code, [])
    
    if not objs:
        return None
    
    btn = widgets.Button(
        description=f"📄 Direkte Einträge ({len(objs)})",
        layout=widgets.Layout(width="auto")
    )
    
    def on_click(b, code_val=code):
        display(Markdown(
            f"### Direkte Objekte für {code_val} – {code_to_name.get(code_val,'')}\n"
            + "\n".join(f"- {o}" for o in objs)
        ))
    
    btn.on_click(on_click)
    return btn

# ----------------------------
# 6️⃣ Baum-Widget generieren
# ----------------------------
def create_tree_widget(code, code_to_objects, tree, agg_counts, direct_counts, code_to_name):
    agg = agg_counts.get(code, 0)
    direct = direct_counts.get(code, 0)
    name = code_to_name.get(code, "")
    
    header = widgets.HTML(
        value=f"<b>{code}</b> – {name} "
              f"<span style='color:gray'>({agg} / {direct})</span>"
    )
    
    elements = [header]
    
    # Direkt-Objekte Button
    direct_btn = direct_objects_button(code, code_to_objects, code_to_name)
    if direct_btn:
        elements.append(direct_btn)
    
    # Kinder
    children = sorted(tree.get(code, []))
    if children:
        child_widgets = [
            create_tree_widget(child, code_to_objects, tree, agg_counts, direct_counts, code_to_name)
            for child in children
        ]
        acc = widgets.Accordion(children=child_widgets)
        for i, c in enumerate(children):
            acc.set_title(i, c)
        elements.append(acc)
    
    return widgets.VBox(elements)

# ----------------------------
# 7️⃣ Filter Dropdown erstellen
# ----------------------------
filter_values = sorted(df.iloc[:, FILTER_COL].dropna().unique().tolist())
filter_dropdown = widgets.Dropdown(
    options=["Alle"] + filter_values,
    description="Filter:"
)
output = widgets.Output()
display(filter_dropdown, output)

# ----------------------------
# 8️⃣ Callback: Baum neu bauen bei Filter
# ----------------------------
def on_filter_change(change):
    with output:
        clear_output()
        
        # Filter anwenden
        if change["new"] == "Alle":
            df_f = df
        else:
            df_f = df[df.iloc[:, FILTER_COL] == change["new"]]
        
        tree_f, agg_f, direct_f, objects_f, code_to_name_f = build_tree_from_df(df_f)
        
        # Wurzelebenen (Codes mit Länge 2)
        roots = sorted([c for c in agg_f if len(c) == 2])
        
        # Baum rendern
        for r in roots:
            display(create_tree_widget(r, objects_f, tree_f, agg_f, direct_f, code_to_name_f))

# Beobachtung aktivieren
filter_dropdown.observe(on_filter_change, names="value")

# Initial anzeigen
on_filter_change({"new": "Alle"})


Dropdown(description='Filter:', options=('Alle', 1, 2, 3, 4), value='Alle')

Output()